In [14]:
import cv2
import numpy as np
from mtcnn import MTCNN
from keras_facenet import FaceNet
import pickle
import mysql.connector
from datetime import datetime, timedelta

# Load the saved SVM model and label encoder
with open('svm_model_with_encoder.pkl', 'rb') as f:
    data = pickle.load(f)
    model = data['model']
    encoder = data['encoder']

# Initialize the FaceNet embedder
embedder = FaceNet()

# Initialize the MTCNN face detector
detector = MTCNN()

# Initialize the database connection
db = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="@Sriram806",
    database="scms"
)
cursor = db.cursor()

# A dictionary to track attendance for each session hour (1 to 8 hours)
session_attendance = {i: set() for i in range(1, 9)}

# Function to get face embedding
def get_embedding(face_img):
    face_img = face_img.astype('float32')
    face_img = np.expand_dims(face_img, axis=0)
    yhat = embedder.embeddings(face_img)
    return yhat[0]

# Function to log attendance into the database
def log_attendance(student_id, timestamp, session_label):
    cursor.execute("INSERT INTO attendance (student_id, timestamp, session) VALUES (%s, %s, %s)", (student_id, timestamp, session_label))
    db.commit()

# Start capturing video from the camera
cap = cv2.VideoCapture(0)

# Get the start time of the first session
session_start_time = datetime.now()
current_session = 1  # Start with session 1 ("1 Hour")

# Live feed loop
while current_session <= 8:  # Loop for up to 8 hours
    ret, frame = cap.read()
    if not ret:
        break
    
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(rgb_frame)
    
    for face in faces:
        x, y, w, h = face['box']
        x, y = abs(x), abs(y)
        face_img = rgb_frame[y:y+h, x:x+w]
        
        try:
            face_img = cv2.resize(face_img, (160, 160))
            embedding = get_embedding(face_img)
            embedding = np.expand_dims(embedding, axis=0)
            prediction = model.predict(embedding)
            probability = model.predict_proba(embedding)
            class_label = encoder.inverse_transform(prediction)[0]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, f'{class_label} ({probability.max()*100:.2f}%)', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
            
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            session_label = f"{current_session} Hour"
            
            # Check if the student has already been logged in the current session
            if class_label not in session_attendance[current_session]:
                # Log the attendance and mark the student as present in this session
                log_attendance(class_label, timestamp, session_label)
                session_attendance[current_session].add(class_label)
        
        except Exception as e:
            print(f"Error processing face: {e}")
            continue
    
    # Display the video feed with the recognized face
    cv2.imshow('Face Recognition Attendance', frame)
    
    # Check if 1 hour has passed since the session started
    if datetime.now() >= session_start_time + timedelta(hours=current_session):
        current_session += 1  # Move to the next session (2 Hour, 3 Hour, ..., 8 Hour)
    
    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
db.close()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 502ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 316ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
